In [3]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
import logging
from pathlib import Path
from typing import Any

from torchvision import tv_tensors

import bb
import bbedit
import tt
import tt_fcos

LOG = logging.getLogger(__name__)
tt.logging_init()

In [ ]:
CATEGORIES = [
    "chicken",
    "cow",
    "creeper",
    "enderman",
    "pig",
    "sheep",
    "skeleton",
    "spider",
    "zombie",
]
CATEGORIES.sort()

DATA_PATH = Path.home() / "data"

# Create dataset with set of images

In [ ]:
new = bb.MCDataset.new(
    dset_dir=DATA_PATH / "new",
    categories=CATEGORIES,
    input_images_dir=DATA_PATH / "minecraft-mobs/images",
)

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Created /Users/joe/data/new, 408 images


# Run new dataset through latest model

In [ ]:
new_dset[0]

(Image([[[126, 126, 126,  ..., 125, 125, 125],
         [126, 126, 126,  ..., 125, 125, 125],
         [126, 126, 126,  ..., 125, 125, 125],
         ...,
         [178, 178, 178,  ..., 196, 196, 196],
         [178, 178, 178,  ..., 196, 196, 196],
         [178, 178, 178,  ..., 196, 196, 196]],
 
        [[185, 185, 185,  ..., 184, 184, 184],
         [185, 185, 185,  ..., 184, 184, 184],
         [185, 185, 185,  ..., 184, 184, 184],
         ...,
         [ 96,  96,  96,  ..., 109, 109, 109],
         [ 96,  96,  96,  ..., 109, 109, 109],
         [ 96,  96,  96,  ..., 109, 109, 109]],
 
        [[255, 255, 255,  ..., 255, 255, 255],
         [255, 255, 255,  ..., 255, 255, 255],
         [255, 255, 255,  ..., 255, 255, 255],
         ...,
         [ 31,  31,  31,  ...,  40,  40,  40],
         [ 31,  31,  31,  ...,  40,  40,  40],
         [ 31,  31,  31,  ...,  40,  40,  40]]], dtype=torch.uint8, ),
 {'image_id': 0,
  'boxes': BoundingBoxes([], size=(0, 4), format=BoundingBoxForma

In [ ]:
import ultralytics as ul

model_path = DATA_PATH / "checkpoints/keep/best.pt"
trainer = tt_fcos.FCOSTrainer.load_checkpoint(model_path, project_dir=model_path.parent, score_thresh=0.5)

new_path = DATA_PATH / "new"
new_dset = bb.MCDataset(new_path)

Loading checkpoint: /Users/joe/data/checkpoints/keep/best.pt
err_keys = <All keys matched successfully>
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


In [46]:
image, target = new_dset[0]
# trainer.plot_infer(image)
target

{'image_id': 0,
 'boxes': BoundingBoxes([], size=(0, 4), format=BoundingBoxFormat.XYXY, canvas_size=(640, 640), clamping_mode=soft),
 'labels': tensor([], dtype=torch.int64)}

In [ ]:
pred = trainer.infer(image)
pred

{'boxes': BoundingBoxes([[230.5681, 231.9958, 306.1765, 389.8882],
                [455.4760, 135.3701, 498.1576, 177.4611],
                [616.8823, 157.6265, 639.3231, 224.5572]], device='mps:0', format=BoundingBoxFormat.XYXY, canvas_size=(640, 640), clamping_mode=soft),
 'scores': tensor([0.7837, 0.6028, 0.5062], device='mps:0'),
 'labels': tensor([2, 1, 1], device='mps:0')}

In [ ]:
# Display results of one query at a time
bb.InferViewer[bb.ImageResult](
    detect_single, new_dset.images, new_dset.categories
).show_widget()

interactive(children=(IntSlider(value=0, continuous_update=False, description='Image:', max=78), Output()), _d…

In [ ]:
for ir in new_dset.iter_images():
    pred_ir = detect_single(ir)
    ir.bboxes = pred_ir.bboxes

In [ ]:
new_dset.save()

# Hand classify

In [ ]:
bbedit.DEBUG.clear_output()
bbedit.DEBUG

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…

In [ ]:
DATA_PATH = Path.home() / "src/data"
new_dset_path = DATA_PATH / "new/info.json"
bbe = bbedit.BBoxEdit(new_dset_path)
bbe.display()

In [ ]:
check_dset = bb.Dataset.load(new_dset_path)
check_dset.view()

interactive(children=(IntSlider(value=0, continuous_update=False, description='Image:', max=78), Output()), _d…

# Merge

In [ ]:
import shutil

import bb

data_dir = Path.home() / "src/data"
input_dirs = [data_dir / "minecraft", data_dir / "new"]
output_dir = data_dir / "mobs_merged"

BBOX_FILE = "info.json"
IMAGE_DIR = "images"


def merge(input_dirs, output_dir):
    output_dir.mkdir(parents=True, exist_ok=True)
    images_subdir = output_dir / IMAGE_DIR
    images_subdir.mkdir(parents=True, exist_ok=True)
    images_rel = Path(IMAGE_DIR)

    new_dset = bb.Dataset(file_path=output_dir / BBOX_FILE)

    categories = None
    file_set = set()
    for input_dir in input_dirs:
        dset = bb.Dataset.load(input_dir / BBOX_FILE)
        if categories is None:
            categories = dset.categories
            new_dset.categories = categories
        assert categories == dset.categories, f"Categories-Mismatch {input_dir}"
        print(f"Input dir={input_dir}, n_files={len(dset.images)}")
        for ir in dset.iter_images():
            fpath = Path(ir.file)
            out_rel = str(images_rel / fpath.name)
            if out_rel in file_set:
                raise FileExistsError(f"Duplicate filename: {out_rel}")

            file_set.add(out_rel)
            new_ir = ir.model_copy(
                update={"file": out_rel}, deep=True
            )
            new_dset.images.append(new_ir)
            shutil.copy2(input_dir / fpath, images_subdir)
    new_dset.save(output_dir / BBOX_FILE)
    print(f"Output dir={output_dir}, n_files={len(list(images_subdir.iterdir()))}")


merge(input_dirs, output_dir)

Input dir=/Users/joe/src/data/minecraft, n_files=329
Input dir=/Users/joe/src/data/new, n_files=79
Output dir=/Users/joe/src/data/mobs_merged, n_files=408


In [ ]:
data_dir = Path.home() / "src/data"
merged_dset = bb.Dataset.load(data_dir / "mobs_merged/info.json")
merged_dset.view()

interactive(children=(IntSlider(value=0, continuous_update=False, description='Image:', max=407), Output()), _…

In [ ]:
# Copy merged into minecraft
# rm -rf minecraft/images minecraft/info.json
# cp -r mobs_merged/images mobs_merged/info.json minecraft

# Stats

In [ ]:
data_dir = Path.home() / "src/data"
dset = bb.Dataset.load(data_dir / "minecraft/info.json")
display(dset.dataset_stats())
display(dset.category_stats())

num_images                    325
num_bboxes                    432
avg_bboxes_per_image     1.329231
num_categories                  9
most_common_category      chicken
least_common_category    skeleton
avg_bbox_width           0.199837
avg_bbox_height          0.265567
avg_bbox_area            0.070426
min_bbox_area             0.00176
max_bbox_area            0.790986
dtype: object

,count,avg_width,avg_height,avg_area
category,,,,
chicken,62,0.1495,0.2031,0.0374
cow,56,0.2146,0.2627,0.0801
pig,54,0.2117,0.1928,0.0551
creeper,50,0.1675,0.3243,0.0706
sheep,47,0.2142,0.2407,0.0677
spider,45,0.3538,0.2497,0.1223
zombie,43,0.1722,0.2955,0.0704
enderman,40,0.1742,0.3994,0.0858
skeleton,35,0.1393,0.2733,0.0565


# Check for small boxes

In [ ]:
# Check for small boxes
import bb

dset_path = DATA_PATH / "minecraft/info.json"

dset = bb.Dataset.load(dset_path)
df = dset.to_df()
for col in ["x1", "y1", "x2", "y2"]:
    df[col] *= 640
df["area"] = (df["x2"] - df["x1"]) * (df["y2"] - df["y1"])
display(df[df["area"] < 512])
display(df[df["area"] < 1024])

In [ ]:
import bb

dset = bb.Dataset.load(dset_path)
dset.category_stats()

In [ ]:
dset.dataset_stats()